In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:

import numpy as np
import pandas as pd

df_n = np.load('drive/MyDrive/cleaned_train_dataset_megafinal_processed.npy', allow_pickle=True)
df = pd.DataFrame(df_n)

In [3]:
df[2].apply(len).mean()

45782.897051941974

In [4]:
df_eval_n = np.load('drive/MyDrive/cleaned_eval_dataset_training.npy', allow_pickle=True)
df_eval = pd.DataFrame(df_eval_n)

In [5]:
df_eval[1].apply(len).mean()

44984.56007751938

In [6]:
import torch
import torch.nn as nn
import transformers
from sentence_transformers import SentenceTransformer
from transformers import DistilBertForSequenceClassification, DistilBertConfig

from sentence_transformers import SentenceTransformer
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Load pre-trained Sentence-BERT model (outputs 768 embeddings)
# sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
sentence_model = SentenceTransformer('all-mpnet-base-v2')

# Load the DistilBert tokenizer (to ensure correct tokenization)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Example long text
long_text = "This is a very long sentence. " * 50  # Make the text longer than 512 tokens for the example


# Define custom embedding layer for the DistilBert model
class CustomSentenceEmbeddingModel(torch.nn.Module):
    def __init__(self, sentence_model):
        super(CustomSentenceEmbeddingModel, self).__init__()
        self.sentence_model = sentence_model

    def forward(self, text):

        sentences = self.divide_tweet(text)
        return self.sentence_model.encode(sentences, convert_to_tensor=True)


    def divide_tweet(self, tweet, num_parts=512):
        """
        Divide a tweet into 512 equal-length parts

        Args:
            tweet (str): Input tweet text
            num_parts (int): Number of parts to divide the tweet into

        Returns:
            list: List of tweet parts
        """
        # Calculate the length of each part
        tweet_length = len(tweet)
        part_length = max(1, tweet_length // num_parts)

        # Divide the tweet into parts
        parts = [
            tweet[i:i+part_length]
            for i in range(0, tweet_length, part_length)
        ]

        # Ensure we have exactly 512 parts, padding or truncating if necessary
        if len(parts) > num_parts:
            parts = parts[:num_parts]
        elif len(parts) < num_parts:
            # Pad with empty strings
            parts.extend([''] * (num_parts - len(parts)))

        return parts

# Load the DistilBert model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
print(model)

# Replace the embeddings layer with the custom Sentence-BERT model
model.distilbert.embeddings = CustomSentenceEmbeddingModel(sentence_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [7]:
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Load pre-trained Sentence-BERT model (outputs 768 embeddings)
sentence_model = SentenceTransformer('all-mpnet-base-v2')

# Load the DistilBert tokenizer (to ensure correct tokenization)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Example long text
long_text = "This is a very long sentence. " * 50  # Make the text longer than 512 tokens for the example

# Define custom embedding layer for the DistilBert model
class CustomSentenceEmbeddingModel(torch.nn.Module):
    def __init__(self, sentence_model):
        super(CustomSentenceEmbeddingModel, self).__init__()
        self.sentence_model = sentence_model

    def forward(self, text):
        sentences = self.divide_tweet(text)
        return self.sentence_model.encode(sentences, convert_to_tensor=True)

    def divide_tweet(self, tweet, num_parts=512):
        """
        Divide a tweet into 512 equal-length parts

        Args:
            tweet (str): Input tweet text
            num_parts (int): Number of parts to divide the tweet into

        Returns:
            list: List of tweet parts
        """
        tweet_length = len(tweet)
        part_length = max(1, tweet_length // num_parts)

        # Divide the tweet into parts
        parts = [
            tweet[i:i+part_length]
            for i in range(0, tweet_length, part_length)
        ]

        # Ensure we have exactly 512 parts, padding or truncating if necessary
        if len(parts) > num_parts:
            parts = parts[:num_parts]
        elif len(parts) < num_parts:
            # Pad with empty strings
            parts.extend([''] * (num_parts - len(parts)))

        return parts

# Load the DistilBert model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Print model architecture (for verification)
print("DistilBert model loaded:", model)

# Replace the embeddings layer with the custom Sentence-BERT model
model.distilbert.embeddings = CustomSentenceEmbeddingModel(sentence_model)


def hook_fn(module, input, output):
    print(f"Module name: {module.__class__.__name__}, Output shape: {output.shape}")

# Assuming 'model' is your PyTorch model
for name, module in model.named_modules():
    module.register_forward_hook(hook_fn)


# Test the chunking function
def test_chunking():
    sentences = model.distilbert.embeddings.divide_tweet(long_text)
    assert len(sentences) == 512, f"Expected 512 parts, got {len(sentences)}"
    print("Chunking test passed. Number of chunks:", len(sentences))

# Test Sentence-BERT embedding output
def test_embedding_output():
    sentences = model.distilbert.embeddings.divide_tweet(long_text)
    embeddings = model.distilbert.embeddings.sentence_model.encode(sentences, convert_to_tensor=True)

    # Check if embeddings are generated and have the correct shape
    assert embeddings.shape == (512, 768), f"Expected embedding shape (512, 768), but got {embeddings.shape}"
    print("Embedding output test passed. Shape:", embeddings.shape)

# Test model forward pass
def test_model_forward():

    sentences = model.distilbert.embeddings.divide_tweet(long_text)
    embeddings = model.distilbert.embeddings.sentence_model.encode(sentences, convert_to_tensor=True)

    # Make a dummy batch of embeddings (add batch dimension)
    batch_embeddings = embeddings.unsqueeze(0)  # Shape: (1, 512, 768)

    # Forward pass through the classification model



    outputs = model.pre_classifier(batch_embeddings)
    print(outputs.shape)
    outputs = model.classifier(outputs)  # Final logits
    assert outputs.shape == (1, 2), f"Expected output shape (1, 2), got {outputs.shape}"
    print("Model forward pass test passed. Logits shape:", outputs.shape)

# Run the tests
test_chunking()
test_embedding_output()
test_model_forward()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBert model loaded: DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropo

AttributeError: 'tuple' object has no attribute 'shape'

In [8]:
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig

# Load pre-trained Sentence-BERT model (outputs 768 embeddings)
sentence_model = SentenceTransformer('all-mpnet-base-v2')

# Load the DistilBert tokenizer (to ensure correct tokenization)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define custom embedding layer for the DistilBert model
import torch
import torch.nn as nn
from transformers import DistilBertForSequenceClassification

# Custom Sentence Embedding Model
class CustomSentenceEmbeddingModel(torch.nn.Module):
    def __init__(self, sentence_model):
        super(CustomSentenceEmbeddingModel, self).__init__()
        self.sentence_model = sentence_model
        self.cls_embedding = nn.Parameter(torch.randn(1, 768))  # learnable CLS embedding

    def forward(self, text):
        sentences = self.divide_tweet(text)
        sentence_embeddings = self.sentence_model.encode(sentences, convert_to_tensor=True).unsqueeze(0)
        # print(sentence_embeddings.shape)
        # Concatenate CLS embedding with sentence embeddings
        # Here, we assume the `sentence_embeddings` is a tensor, and we want to add the CLS token to it
        cls_embeddings = self.cls_embedding.expand(sentence_embeddings.size(0), -1).unsqueeze(0)  # Match the batch size
        # print(cls_embeddings.shape)
        sentence_embeddings_with_cls = torch.cat([cls_embeddings, sentence_embeddings], dim=1)  # Concatenate along the feature axis
        # print(sentence_embeddings_with_cls.shape)
        return sentence_embeddings_with_cls # Freeze to avoid updatessentence_embeddings_with_cls

    def divide_tweet(self, tweet, num_parts=511):
        """
        Divide a tweet into 512 equal-length parts

        Args:
            tweet (str): Input tweet text
            num_parts (int): Number of parts to divide the tweet into

        Returns:
            list: List of tweet parts
        """
        tweet_length = len(tweet)
        part_length = max(1, tweet_length // num_parts)

        parts = [
            tweet[i:i + part_length]
            for i in range(0, tweet_length, part_length)
        ]

        if len(parts) > num_parts:
            parts = parts[:num_parts]
        elif len(parts) < num_parts:
            parts.extend([''] * (num_parts - len(parts)))

        return parts

class CustomDistilBERTModel(nn.Module):
    def __init__(self, sentence_model, num_labels=2):
        super(CustomDistilBERTModel, self).__init__()

        # Load DistilBERT pre-trained model for sequence classification
        self.distilbert = DistilBertForSequenceClassification.from_pretrained(
            "distilbert-base-uncased", num_labels=num_labels
        )

        # Replace the embeddings layer with nn.Identity
        self.distilbert.distilbert.embeddings = nn.Identity()

        # Replace the embeddings layer with the custom Sentence-BERT model
        self.new_embeds = CustomSentenceEmbeddingModel(sentence_model)

        # Override the classifier
        self.classifier = nn.Linear(self.distilbert.config.hidden_size, num_labels)  # Adjusted for no extra CLS embedding

    def forward(self, text, labels):
        # Get the custom embeddings from the Sentence-BERT model
        embeddings = self.new_embeds(text).long()
        # print("Embeddings shape:", embeddings.shape)

        # Pass embeddings through the transformer layers of DistilBERT
        outputs = self.distilbert.distilbert(embeddings)  # Get output from DistilBERT transformer layers
        embeddings = outputs.last_hidden_state  # Extract hidden states from the transformer

        # print("DistilBERT transformer output shape:", embeddings.shape)

        # Use the CLS token representation directly from the transformer output
        cls_representation = embeddings[:, 0, :]  # First token (CLS token) from transformer output

        # Pass the CLS representation through the classifier
        logits = self.classifier(cls_representation)

        return logits


class CustomDistilBERTModelSimple(nn.Module):
    def __init__(self, sentence_model, num_labels=2):
        super(CustomDistilBERTModelSimple, self).__init__()

        # Load DistilBERT pre-trained model for sequence classification
        self.distilbert = DistilBertForSequenceClassification.from_pretrained(
            "distilbert-base-uncased", num_labels=num_labels
        )

        # Replace the embeddings layer with nn.Identity
        # self.distilbert.distilbert.embeddings = nn.Identity()

        # Replace the embeddings layer with the custom Sentence-BERT model
        self.new_embeds = CustomSentenceEmbeddingModel(sentence_model)

        # Override the classifier
        self.classifier = nn.Linear(self.distilbert.config.hidden_size, num_labels)  # Adjusted for no extra CLS embedding

    def forward(self, text, labels=None):
        # Get the custom embeddings from the Sentence-BERT model
        embeddings = self.new_embeds(text).long()
        # print("Embeddings shape:", embeddings.shape)

        B, L, T = embeddings.shape

        # Pass embeddings through the transformer layers of DistilBERT
        input_ids = torch.arange(L).unsqueeze(0).repeat(B, 1)
        outputs = self.distilbert(inputs_embeds = embeddings, labels=labels)  # Get output from DistilBERT transformer layers
        # embeddings = outputs.last_hidden_state  # Extract hidden states from the transformer
        # embeddings = outputs

        # Use the CLS token representation directly from the transformer output
        # cls_representation = embeddings[:, 0, :]  # First token (CLS token) from transformer output

        # Pass the CLS representation through the classifier
        # logits = self.classifier(cls_representation)
        # print(outputs)
        logits = outputs.logits
        return outputs


sentence_model = SentenceTransformer('all-mpnet-base-v2')
# Instantiate the custom model
model = CustomDistilBERTModelSimple(sentence_model)

# Example input
long_text = "This is a very long sentence. " * 50 # Make the text longer than 512 tokens for the example
# text = ["This is a ve", "awfdawfawf"]
# inputs = tokenizer(long_text, return_tensors="pt", truncation=True, padding=True)

# # Forward pass through the custom model
# output = model(long_text, torch.tensor([1]))
# print(output)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, texts, labels, max_length=512):
        self.texts = texts
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Tokenize the text and pad/truncate it to the max length
        # We need to squeeze out the extra dimension that `return_tensors="pt"` adds
        # input_ids = encoding["input_ids"].squeeze(0)  # shape (seq_length,)
        # attention_mask = encoding["attention_mask"].squeeze(0)  # shape (seq_length,)

        return text, label

# Step 3: Create the Dataset
dataset = TextDataset(df[2], df[1])

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from tqdm.auto import tqdm  # Added tqdm import

import torch.optim as optim
import torch.nn as nn
import random

class TweetSentenceClassifier:
    def __init__(self, model_name='distilbert-base-uncased', sentence_model=SentenceTransformer('all-mpnet-base-v2'), num_labels=2):
        """
        Initialize the Tweet Sentence Classifier

        :param model_name: Pretrained model to use as base
        :param num_labels: Number of classification categories
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        # If num_labels is not specified, it will be inferred from the dataset
        self.num_labels = num_labels
        self.model = CustomDistilBERTModelSimple(sentence_model).to(self.device)

    def prepare_dataset(self, df, text_column, label_column, test_size=0.2, random_state=42):
        """
        Prepare dataset for training

        :param df: Pandas DataFrame containing tweets
        :param text_column: Name of column with tweet text
        :param label_column: Name of column with labels
        :param test_size: Proportion of dataset to use for testing
        :param random_state: Random seed for reproducibility
        :return: Tuple of train and test datasets
        """
        # Infer number of labels if not specified
        if self.num_labels is None:
            self.num_labels = len(df[label_column].unique())

        # Initialize model with correct number of labels


        # Tokenization function
        def tokenize_function(text):
            return self.tokenizer(
                text,
                padding='max_length',
                truncation=True,
                max_length=512
            )

        # Split the data
        train_df, test_df = train_test_split(
            df,
            test_size=test_size,
            random_state=random_state,
        )

        train_df.reset_index(drop=True, inplace=True)
        test_df.reset_index(drop=True, inplace=True)

        # Tokenize datasets
        # train_df['tokenized'] = train_df[text_column].apply(tokenize_function)
        # test_df['tokenized'] = test_df[text_column].apply(tokenize_function)
        self.test_data = TextDataset(test_df[text_column], train_df[label_column])
        self.train_data = TextDataset(train_df[text_column], train_df[label_column])
        # return train_df, test_df

    def prepare_general_dataset(self, df, text_column, label_column, test_size=0.2, random_state=42):
        """
        Prepare general dataset for training

        :param df: Pandas DataFrame containing tweets
        :param text_column: Name of column with tweet text
        :param label_column: Name of column with labels
        :param test_size: Proportion of dataset to use for testing
        :param random_state: Random seed for reproducibility
        :return: Tuple of train and test datasets
        """
        # Infer number of labels if not specified
        if self.num_labels is None:
            self.num_labels = len(df[label_column].unique())


        # Tokenization function
        # def tokenize_function(text):
        #     return self.tokenizer(
        #         text,
        #         truncation=True,
        #         max_length=len(text)*10
        #     )

        # Split the data
        train_df, test_df = train_test_split(
            df,
            test_size=test_size,
            random_state=random_state,
        )

        # Tokenize datasets
        # train_df['tokenized'] = train_df[text_column].apply(tokenize_function)
        # test_df['tokenized'] = test_df[text_column].apply(tokenize_function)

        return train_df, test_df


    def prepare_loaders_from_generals(self, train_df, test_df, label_column, max_length=512):
        def process_tokens(tokenized_items):
            processed_input_ids = []
            processed_attention_masks = []

            for item in tokenized_items:
                input_ids = item['input_ids']
                attention_mask = item['attention_mask']

                # If sequence is shorter than 512, pad
                if len(input_ids) <= max_length:
                    pad_length = max_length - len(input_ids)
                    padded_input_ids = input_ids + [self.tokenizer.pad_token_id] * pad_length
                    padded_attention_mask = attention_mask + [0] * pad_length
                    processed_input_ids.append(padded_input_ids)
                    processed_attention_masks.append(padded_attention_mask)

                # If sequence is longer than 512, choose random start
                else:
                    # Calculate the maximum possible start index
                    max_start_index = len(input_ids) - max_length

                    # Choose a random start index
                    start_index = random.randint(0, max_start_index)

                    # Extract 512 consecutive tokens
                    windowed_input_ids = input_ids[start_index:start_index + max_length]
                    windowed_attention_mask = attention_mask[start_index:start_index + max_length]

                    processed_input_ids.append(windowed_input_ids)
                    processed_attention_masks.append(windowed_attention_mask)

            return processed_input_ids, processed_attention_masks

        # Ensure random seed for reproducibility
        # random.seed(42)

        # Process train and test data
        train_input_ids, train_attention_mask = process_tokens(train_df['tokenized'])
        test_input_ids, test_attention_mask = process_tokens(test_df['tokenized'])

        # Convert to tensors
        train_input_ids = torch.tensor(train_input_ids)
        train_attention_mask = torch.tensor(train_attention_mask)
        # train_labels = torch.tensor(train_df[label_column].values)

        test_input_ids = torch.tensor(test_input_ids)
        test_attention_mask = torch.tensor(test_attention_mask)
        # test_labels = torch.tensor(test_df[label_column].values)

        train_labels = torch.tensor(np.array(train_df[label_column].to_list(), dtype=np.int64), dtype=torch.long)
        test_labels = torch.tensor(np.array(test_df[label_column].to_list(), dtype=np.int64), dtype=torch.long)

        # Create data loaders
        train_dataset = torch.utils.data.TensorDataset(
            train_input_ids,
            train_attention_mask,
            train_labels
        )
        test_dataset = torch.utils.data.TensorDataset(
            test_input_ids,
            test_attention_mask,
            test_labels
        )

        train_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=32,
            shuffle=False
        )
        test_loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=32,
            shuffle=False
        )

        return train_loader, test_loader


    def compute_metrics(self, pred):
        """
        Compute evaluation metrics

        :param pred: Predictions from the model
        :return: Dictionary of metrics
        """
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)

        # Compute classification report
        report = classification_report(labels, preds, output_dict=True)

        return {
            'accuracy': report['accuracy'],
            'macro_f1': report['macro avg']['f1-score'],
            'weighted_f1': report['weighted avg']['f1-score']
        }

    def custom_loader_train(self, train_df, test_df, label_column, epochs=1, learning_rate=2e-5):
        """
        Custom training method with tqdm progress bars

        :param train_df: Training dataframe with tokenized column
        :param test_df: Test dataframe with tokenized column
        :param label_column: Name of the label column
        :param epochs: Number of training epochs
        :param learning_rate: Learning rate for optimizer
        """

        # test_input_ids = torch.tensor([x['input_ids'] for x in test_df['tokenized']])
        # test_attention_mask = torch.tensor([x['attention_mask'] for x in test_df['tokenized']])
        # test_labels = torch.tensor(test_df[label_column].values)

        # # Create data loaders

        # test_dataset = torch.utils.data.TensorDataset(
        #     test_input_ids,
        #     test_attention_mask,
        #     test_labels
        # )

        # test_loader = torch.utils.data.DataLoader(
        #     test_dataset,
        #     batch_size=32,
        #     shuffle=False
        # )



        # Prepare optimizer and loss
        optimizer = optim.AdamW(
            self.model.parameters(),
            lr=learning_rate
        )
        criterion = nn.CrossEntropyLoss()

        # Training loop with tqdm progress bars
        for epoch in range(epochs):
            # Training phase
            self.model.train()
            train_loss = 0
            train_correct = 0
            train_total = 0

            train_loader, test_loader = self.prepare_loaders_from_generals(train_df, test_df, label_column)


            # Wrap train_loader with tqdm for progress bar
            train_progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} (Training)", leave=False)
            for batch in train_progress_bar:
                # Unpack batch
                input_ids, attention_mask, labels = [b.to(self.device) for b in batch]

                # Zero gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )
                loss = outputs.loss
                logits = outputs.logits

                # Backward pass
                loss.backward()
                optimizer.step()

                # Compute accuracy
                train_loss += loss.item()
                _, predicted = torch.max(logits, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()

                # Update progress bar
                train_progress_bar.set_postfix({
                    'Loss': f'{loss.item():.4f}',
                    'Accuracy': f'{100 * train_correct / train_total:.2f}%'
                })

            # Validation phase
            self.model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0
            all_preds = []
            all_labels = []

            # Wrap test_loader with tqdm for progress bar
            val_progress_bar = tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} (Validation)", leave=False)
            with torch.no_grad():
                for batch in val_progress_bar:
                    input_ids, attention_mask, labels = [b.to(self.device) for b in batch]

                    outputs = self.model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels
                    )
                    loss = outputs.loss
                    logits = outputs.logits

                    val_loss += loss.item()
                    _, predicted = torch.max(logits, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()

                    all_preds.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

                    # Update progress bar
                    val_progress_bar.set_postfix({
                        'Loss': f'{loss.item():.4f}',
                        'Accuracy': f'{100 * val_correct / val_total:.2f}%'
                    })

            # Print epoch statistics
            # print(f"Epoch {epoch+1}/{epochs}")
            # print(f"Train Loss: {train_loss/len(train_loader):.4f}")
            # print(f"Train Accuracy: {100 * train_correct / train_total:.2f}%")
            # print(f"Val Loss: {val_loss/len(test_loader):.4f}")
            # print(f"Val Accuracy: {100 * val_correct / val_total:.2f}%")

            # Compute detailed classification report
            # report = classification_report(all_labels, all_preds)
            # print("Classification Report:")
            # print(report)

        return self.model

    def custom_train(self, text_column, label_column, epochs=1, learning_rate=2e-5):
        """
        Custom training method with tqdm progress bars

        :param train_df: Training dataframe with tokenized column
        :param test_df: Test dataframe with tokenized column
        :param label_column: Name of the label column
        :param epochs: Number of training epochs
        :param learning_rate: Learning rate for optimizer
        """
        # Prepare data tensors
        # print(train_df[text_column].shape)
        # train_input_ids = train_df[text_column]
        # train_attention_mask = torch.tensor([x['attention_mask'] for x in train_df['tokenized']])
        # train_labels = torch.tensor(train_df[label_column].values)

        # test_input_ids = test_df[text_column]
        # test_attention_mask = torch.tensor([x['attention_mask'] for x in test_df['tokenized']])
        # test_labels = torch.tensor(test_df[label_column].values)


        # train_labels = torch.tensor(np.array(train_df[label_column].to_list(), dtype=np.int64), dtype=torch.long)
        # test_labels = torch.tensor(np.array(test_df[label_column].to_list(), dtype=np.int64), dtype=torch.long)

        # Create data loaders
        # train_dataset = torch.utils.data.TensorDataset(
        #     train_input_ids,
        #     # train_attention_mask,
        #     train_labels
        # )
        # test_dataset = torch.utils.data.TensorDataset(
        #     test_input_ids,
        #     # test_attention_mask,
        #     test_labels
        # )

        train_loader = torch.utils.data.DataLoader(
            self.train_data,
            batch_size=1,
            shuffle=True
        )
        test_loader = torch.utils.data.DataLoader(
            self.test_data,
            batch_size=1,
            shuffle=True
        )

        # Prepare optimizer and loss
        optimizer = optim.AdamW(
            self.model.parameters(),
            lr=learning_rate
        )
        criterion = nn.CrossEntropyLoss()

        # Training loop with tqdm progress bars
        for epoch in range(epochs):
            # Training phase
            self.model.train()
            train_loss = 0
            train_correct = 0
            train_total = 0

            # Wrap train_loader with tqdm for progress bar
            train_progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} (Training)", leave=False)
            for batch in train_progress_bar:
                # Unpack batch
                input_ids, labels = [b for b in batch]
                # print(len(input_ids))

                labels = labels.to(self.device)

                # Zero gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = self.model(
                    input_ids[0],
                    # attention_mask=attention_mask,
                    labels=labels[0].to(self.device)
                )
                loss = outputs.loss
                logits = outputs.logits

                # Backward pass
                loss.backward()
                optimizer.step()

                # Compute accuracy
                train_loss += loss.item()
                _, predicted = torch.max(logits, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()

                # Update progress bar
                train_progress_bar.set_postfix({
                    'Loss': f'{loss.item():.4f}',
                    'Accuracy': f'{100 * train_correct / train_total:.2f}%'
                })

            # Validation phase
            self.model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0
            all_preds = []
            all_labels = []

            # Wrap test_loader with tqdm for progress bar
            val_progress_bar = tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} (Validation)", leave=False)
            with torch.no_grad():
                for batch in val_progress_bar:
                    input_ids, labels = [b for b in batch]


                    labels = labels.to(self.device)

                    outputs = self.model(input_ids,
                        # attention_mask=attention_mask,
                        labels=labels[0]
                    )
                    loss = outputs.loss
                    logits = outputs.logits

                    val_loss += loss.item()
                    _, predicted = torch.max(logits, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()

                    all_preds.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

                    # Update progress bar
                    val_progress_bar.set_postfix({
                        'Loss': f'{loss.item():.4f}',
                        'Accuracy': f'{100 * val_correct / val_total:.2f}%'
                    })

            # Print epoch statistics
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"Train Loss: {train_loss/len(train_loader):.4f}")
            print(f"Train Accuracy: {100 * train_correct / train_total:.2f}%")
            print(f"Val Loss: {val_loss/len(test_loader):.4f}")
            print(f"Val Accuracy: {100 * val_correct / val_total:.2f}%")

            # Compute detailed classification report
            # report = classification_report(all_labels, all_preds)
            print("Classification Report:")
            # print(report)

        return self.model

    def predict(self, texts, start=0):
        """
        Make predictions on new texts

        :param texts: List of text strings to classify
        :return: Predictions and their probabilities
        """
        # Tokenize inputs
        inputs = self.tokenizer(
            texts[start:],
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        ).to(self.device)

        # Get model predictions
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)
            predictions = torch.argmax(probs, dim=1)

        return predictions.cpu().numpy(), probs.cpu().numpy()


    def freeze(self):
        for name, param in self.model.named_parameters():
            if 'classifier' not in name and 'cls_embedding' not in name and 'transformer' not in name:
                param.requires_grad = False

    def predict_iterate(self, texts):
        """
        Make predictions on new texts

        :param texts: List of text strings to classify
        :return: Predictions and their probabilities
        """
        # Tokenize inputs
        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        ).to(self.device)

        # Get model predictions
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)
            predictions = torch.argmax(probs, dim=1)

        return predictions.cpu().numpy(), probs.cpu().numpy()

In [11]:
# Initialize classifier
classifier = TweetSentenceClassifier()  # Specify number of labels

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
classifier.freeze()

In [13]:
# Prepare dataset (adjust column names as needed)
classifier.prepare_dataset(
    df,
    text_column=2,
    label_column=1
)

In [14]:
for name, param in classifier.model.named_parameters():
    if param.requires_grad:
        print(f"Trainable parameter: {name}")
    else:
        print(f"Non-trainable parameter: {name}")

Non-trainable parameter: distilbert.distilbert.embeddings.word_embeddings.weight
Non-trainable parameter: distilbert.distilbert.embeddings.position_embeddings.weight
Non-trainable parameter: distilbert.distilbert.embeddings.LayerNorm.weight
Non-trainable parameter: distilbert.distilbert.embeddings.LayerNorm.bias
Trainable parameter: distilbert.distilbert.transformer.layer.0.attention.q_lin.weight
Trainable parameter: distilbert.distilbert.transformer.layer.0.attention.q_lin.bias
Trainable parameter: distilbert.distilbert.transformer.layer.0.attention.k_lin.weight
Trainable parameter: distilbert.distilbert.transformer.layer.0.attention.k_lin.bias
Trainable parameter: distilbert.distilbert.transformer.layer.0.attention.v_lin.weight
Trainable parameter: distilbert.distilbert.transformer.layer.0.attention.v_lin.bias
Trainable parameter: distilbert.distilbert.transformer.layer.0.attention.out_lin.weight
Trainable parameter: distilbert.distilbert.transformer.layer.0.attention.out_lin.bias
Tr

In [17]:
# Train the model
model = classifier.custom_train(2, 1, 2, learning_rate=10e-7)

Epoch 1/2 (Training):   0%|          | 0/1709 [00:00<?, ?it/s]

Epoch 1/2 (Validation):   0%|          | 0/428 [00:00<?, ?it/s]

IndexError: tuple index out of range

In [60]:
classifier.model("awdawd")

TypeError: CustomDistilBERTModelSimple.forward() missing 1 required positional argument: 'labels'